In [3]:
from Learning import *
from Graphing import *

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\alfew\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.


In [11]:
models, scorers = models_c, scorers_c[:1]
Y_labels = Y_labels_de[1:] # All except 'inc', which is binary
# Y_labels = Y_labels_de
# Y_labels = ['dif', 'nrd', 'skt'] # Sketchability
# Y_labels = ['dif', 'nrd', 'skt', 'vis'] # Visuality
# Y_labels = ['vis', 'phy', 'obj'] # Physicality
df, X, Y = load_data(Y_labels)

# Convert for classification
threshold = 5.0
Y = (Y > threshold).astype(np.int)

# Class balance percentages
[round(100 * max(np.mean(Y[:, i]), 1- np.mean(Y[:, i])), 2) for i in range(len(Y_labels))]

[76.92, 63.46, 55.77, 67.31, 76.92, 53.85, 73.08]

In [ ]:
# Train Multi-task models on interval_size-sample increments of data, up to the whole dataset
interval_size = 26
ns_samples = list(np.arange(0, X.shape[0], interval_size)[1:-1]) + [ X.shape[0] ]

# Take ns_eval_samples samples of random x for each increment, to get accurate results at low n
ns_eval_samples = [10] * len(ns_samples)
print(sum(sum(Y == 1))/sum(sum(np.ones(Y.shape))))
X.shape, Y.shape, ns_samples, ns_eval_samples

In [10]:
# Perform cross validation prediction
Y_pr = do_cv_learning_curve(X, Y, models, ns_samples, ns_eval_samples, cv=3) # Result includes sample indices used
save_ld((X, Y, Y_pr, Y_labels, models, scorers, ns_samples, ns_eval_samples), "preds_52_" + str(time.time()))
# X, Y, Y_pr, Y_labels, models, scorers, ns_samples, ns_eval_samples = load_learning_data("preds_s_1e_1513502596.5260706")
len(Y_pr), len(Y_pr[-1]), len(Y_pr[-1][-1]), Y_pr[-1][-1][-1][0].shape

(7, 1, 10, (52, 7))

In [11]:
# Evaluate predictions
scores = score_learning_curve(Y, Y_pr, models, ns_samples, ns_eval_samples, scorers)
print_learning_curve_mean_scores(models, scorers, scores, Y_labels, ["skt"]); print()
# graph_learning_curve(models, scorers, scores, ns_samples, Y_labels)

	AdaBoostClassifier
'multitask_acc: [0.767]'
	RandomForestClassifier
'multitask_acc: [0.771]'
	DecisionTreeClassifier
'multitask_acc: [0.794]'
	KNeighborsClassifier
'multitask_acc: [0.6]'
	SVC
'multitask_acc: [0.735]'
	GaussianProcessClassifier
'multitask_acc: [0.744]'
	LogisticRegression
'multitask_acc: [0.783]'



In [15]:
# For debug: convert x data into random numbers & try to learn
X = np.random.random((X.shape[0], 1000))
Y_pr = do_cv_learning_curve(X, Y, models, ns_samples, ns_eval_samples, cv=3) # Result includes sample indices used
len(Y_pr), len(Y_pr[-1]), len(Y_pr[-1][-1]), Y_pr[-1][-1][-1][0].shape

(7, 1, 10, (52, 7))

In [16]:
# Evaluate predictions
scores = score_learning_curve(Y, Y_pr, models, ns_samples, ns_eval_samples, scorers)
print_learning_curve_mean_scores(models, scorers, scores, Y_labels, ["skt"]); print()
# graph_learning_curve(models, scorers, scores, ns_samples, Y_labels)

	AdaBoostClassifier
'multitask_acc: [0.713]'
	RandomForestClassifier
'multitask_acc: [0.744]'
	DecisionTreeClassifier
'multitask_acc: [0.733]'
	KNeighborsClassifier
'multitask_acc: [0.592]'
	SVC
'multitask_acc: [0.692]'
	GaussianProcessClassifier
'multitask_acc: [0.777]'
	LogisticRegression
'multitask_acc: [0.75]'



In [17]:
# Inspect errors
print("Word: '" + df.index[11] + "'")
print("Prediction: " + str([round(y, 3) for y in results[0][-1][0][0][11]]))
print("Actual: " + str(Y[11]))

Word: 'bounces'


NameError: name 'results' is not defined

In [ ]:
# Load data for new word prediction
ngrams_db = load_ngram_counts()             # Load ngram counts database
wrep_lib = WordRepLibrary(lib_paths[-1])    # Load word representation database

In [ ]:
# Predict for some random new words
n_pred = 100
i_seen = map(int, list(df["lib_i"]))
model = models_lib[models[-1]]
model.fit(X, Y)
print("         ", Y_labels)
for i in range(n_pred):
    word, word_i, x = wrep_lib.get_new_word(i_seen)
    while word not in ngrams_db.index or not valid_word(word, verbose=False):
        word, word_i, x = wrep_lib.get_new_word()
    x += list(ngrams_db.loc[word]) + [ google_search_count(word) ]
    y_pred = model.predict(np.atleast_2d(x))
    print(word + ": " + str(format_numlist(y_pred[0], 3)))